In [8]:
# importing libraries
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'webster-data445-bucket'
bucket = s3.Bucket(bucket_name)

# defining the csv file
file_key = 'telecom_churn.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading the csv file
churn_data = pd.read_csv(file_content_stream)
# dropping missing values
churn_data = churn_data.dropna()
churn_data.head()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.7,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.7,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.0,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.0,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.0,3,166.7,113,41.0,7.42,10.1


In [10]:
# relative frequency for churn
churn_freq = churn_data['Churn'].value_counts(normalize = True) 
churn_freq

0    0.855086
1    0.144914
Name: Churn, dtype: float64

In [12]:
X = churn_data[['AccountWeeks', 'ContractRenewal', 'CustServCalls', 'MonthlyCharge', 'DayMins']]
Y = churn_data['Churn']

# splitting into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [24]:
# building the random forest
RF1 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

# predicting on test
RF_pred = RF1.predict_proba(X_test)[:, 1]

# computing ROC curve
fpr, tpr, threshold = roc_curve(Y_test, RF_pred)

# creating a dataframe with unpacked roc values
cutoffs = pd.DataFrame({'False_Positive' : fpr, 'True_Positive' : tpr, 'Cutoff' : threshold})
cutoffs.head(10)

,False_Positive,True_Positive,Cutoff
0,0.000000,0.000000,1.684444
1,0.000000,0.010309,0.684444
2,0.000000,0.175258,0.586851
3,0.003509,0.175258,0.581000
4,0.003509,0.237113,0.557883
5,0.005263,0.237113,0.556930
6,0.005263,0.402062,0.510899
7,0.007018,0.402062,0.507773
8,0.007018,0.443299,0.469004
9,0.008772,0.443299,0.463909


In [38]:
# finding the optimal cutoff
cutoffs['True_Positive_minus_1'] = cutoffs['True_Positive'] - 1

# finding euclidean distance
cutoffs['Euclidian_dist'] = np.sqrt(cutoffs['False_Positive']**2 + cutoffs['True_Positive_minus_1']**2)
cutoffs = cutoffs.sort_values(by = 'Euclidian_dist').reset_index(drop = True)

# changing likelihoods to labels
RF_pred = np.where(RF_pred < cutoffs['Euclidian_dist'][0], 0, 1)

# printing classification report
print(classification_report(Y_test, RF_pred))

              precision    recall  f1-score   support

           0       0.95      0.86      0.90       570
           1       0.46      0.71      0.56        97

    accuracy                           0.84       667
   macro avg       0.70      0.79      0.73       667
weighted avg       0.88      0.84      0.85       667

